In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('SW.xlsx',sheet_name='Budget')

### Расчеты до изменений 

In [ ]:
# первоначальный расчет до изменений
df['salary'] = df['Base_monthly_salary'] * 5 # зп до натбавки
df['salary'] = df['salary'] + (df['Base_monthly_salary']*(1 + df['salary_increase_1Jun'])) * 7 # зп после прибавки в июне
df['bonus'] = df['salary'] * df['bonus_year']
df['spec_bonus'] = df['salary'] * df['specific_bonus_year']
df['salary_per_day'] = round(df['salary'] / 248,2)
df['vacation_balance'] = df['salary_per_day'] * df['End_vacation']
df['All_budget'] = df['salary'] + df['bonus'] + df['spec_bonus'] + df['vacation_balance'] + (df['car_month']*12) + df['Med_year']

### Расчет с учетом изменений 

#### 2. New contract with Insurance company assumes annual cost decline for grades 2 and 5 Operations and Marketing by 30% and cost increase for all Central jobs by 15%. For the rest, no change in insurance cost.Changes apply since 1 September

In [ ]:
df['Med_year'] = np.where((df['Job_grade'] == 2) & (df['Function'] == "Operations"),((df['Med_year']/12)*8 + ((df['Med_year']*0.7)/12)*4),df['Med_year'])
df['Med_year'] = np.where((df['Job_grade'] == 2) & (df['Function'] == "Marketing"),((df['Med_year']/12)*8 + ((df['Med_year']*0.7)/12)*4),df['Med_year'])
df['Med_year'] = np.where((df['Job_grade'] == 5) & (df['Function'] == "Operations"),((df['Med_year']/12)*8 + ((df['Med_year']*0.7)/12)*4),df['Med_year'])
df['Med_year'] = np.where((df['Job_grade'] == 5) & (df['Function'] == "Marketing"),((df['Med_year']/12)*8 + ((df['Med_year']*0.7)/12)*4),df['Med_year'])
df['Med_year'] = np.where((df['Function'] == "Central"),((df['Med_year']/12)*8 + ((df['Med_year']*1.15)/12)*4), df['Med_year'])

#### 3. Year End vacation balance is expected to increase by 2 days for all employees in Marketing, while Central employees grade 8 will reduce theirs by half.For the rest, no change vacation balance

In [ ]:
df['End_vacation'] = np.where((df['Function'] == "Marketing"),df['End_vacation']+2, df['End_vacation'])
df['End_vacation'] = np.where((df['Function'] == "Central"),df['End_vacation']/2,df['End_vacation'])

#### 4. To stay competitive in terms of pay, additional salary increase was introduced in October. 5% for grades 2,4 and 2% the rest

In [ ]:

df['salary'] = df['Base_monthly_salary'] * 5 # зп до натбавки

df['Base_monthly_salary'] = (df['Base_monthly_salary']*(1 + df['salary_increase_1Jun']))
df['salary'] = df['salary'] + (df['Base_monthly_salary']* 7) # зп после прибав


In [ ]:
# прибавка salary с октября 
df['Base_monthly_salary'] = np.where(((df['Job_grade'] == 2) | (df['Job_grade'] == 4)),df['Base_monthly_salary']*1.05,df['Base_monthly_salary']*1.02)

In [ ]:
# изменение salary с учетеом прибавки
df['salary'] += np.where(((df['Job_grade'] == 2) | (df['Job_grade'] == 4)),(df['Base_monthly_salary']-(df['Base_monthly_salary']/1.05))*3, \
                                                                           (df['Base_monthly_salary']-(df['Base_monthly_salary']/1.02))*3)

#### 5. Budget assumed no overtimes, however, due to production volume increase, Operations grades 2 and 4 worked 90 hours each on top.Standard working hours: 10 hours/day, 5 days a week. Overtimes are paid x2 stardard salary

In [ ]:
#переработки (выплачиваются в конце года)
df['overtime_payment'] = 0
df['overtime_payment'] = np.where((df['Job_grade'] == 2) & (df['Function'] == "Operations"),round(((df['salary']/248)*2)*9,2),df['overtime_payment'])
df['overtime_payment'] = np.where((df['Job_grade'] == 4) & (df['Function'] == "Operations"),round(((df['salary']/248)*2)*9,2),df['overtime_payment'])

#### 6.Overall, company is expecting to deliver strong financial results, so company bonus will be higher by 50%

In [ ]:
df['bonus_year'] = df['bonus_year']*1.5

In [ ]:
df['bonus'] = df['salary'] * df['bonus_year']
df['spec_bonus'] = df['salary'] * df['specific_bonus_year']
df['salary_per_day'] = round(df['salary'] / 248,2)
df['vacation_balance'] = df['salary_per_day'] * df['End_vacation']
df['All_budget'] = df['salary'] + df['bonus'] + df['spec_bonus'] + df['vacation_balance'] + (df['car_month']*12) + df['Med_year']+df['overtime_payment']

In [ ]:
df.to_excel('res.xlsx')